In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['font.family'] = 'NanumGothic'

import os, sys, random
from pathlib import Path

from tqdm import tqdm
from fastprogress import progress_bar

import pickle

import warnings
warnings.filterwarnings(action='ignore')

In [2]:
CUR_DIR = Path.cwd()
DATA_DIR = CUR_DIR / 'data/preprocessed'
DATA_DIR

WindowsPath('C:/Users/안현준/풍년보험/data/preprocessed')

In [3]:
df_crops = pd.read_csv('./results_LGBM/predict_prices_mape.csv', encoding='utf-8-sig')
df_crops

,Unnamed: 0,crop_id,d,price,price_lag_12,price_lag_13,price_lag_14,price_lag_15,price_lag_16,price_lag_17,...,moving_std_30,moving_avg_36,moving_std_36,price_momentum,연월,year,month,season,cluster_id,pred_price
0,0,0,1,12191.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,2001_1,2001,1,3,0,12671.840043
1,1,0,2,11558.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,2001_2,2001,2,3,0,14223.632715
2,2,0,3,11885.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,2001_3,2001,3,1,0,14658.669814
3,3,0,4,12133.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,2001_4,2001,4,1,0,16048.912606
4,4,0,5,22625.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,2001_5,2001,5,1,0,16032.049901
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8059,8059,31,248,NaN,51334.0,45886.0,42578.0,40602.0,47785.0,62178.0,...,12216.0,49540.0,13530.0,0,2021_8,2021,8,2,1,51229.414671
8060,8060,31,249,NaN,92749.0,51334.0,45886.0,42578.0,40602.0,47785.0,...,14480.0,50400.0,15224.0,1,2021_9,2021,9,0,1,70389.648956
8061,8061,31,250,NaN,63283.0,92749.0,51334.0,45886.0,42578.0,40602.0,...,14536.0,51230.0,15090.0,0,2021_10,2021,10,0,1,40482.972822
8062,8062,31,251,NaN,49019.0,63283.0,92749.0,51334.0,45886.0,42578.0,...,14424.0,51550.0,14920.0,0,2021_11,2021,11,0,1,51179.942012


In [4]:
df = df_crops[['crop_id', '연월', 'year', 'month', 'price', 'pred_price']]
df

,crop_id,연월,year,month,price,pred_price
0,0,2001_1,2001,1,12191.0,12671.840043
1,0,2001_2,2001,2,11558.0,14223.632715
2,0,2001_3,2001,3,11885.0,14658.669814
3,0,2001_4,2001,4,12133.0,16048.912606
4,0,2001_5,2001,5,22625.0,16032.049901
...,...,...,...,...,...,...
8059,31,2021_8,2021,8,NaN,51229.414671
8060,31,2021_9,2021,9,NaN,70389.648956
8061,31,2021_10,2021,10,NaN,40482.972822
8062,31,2021_11,2021,11,NaN,51179.942012


In [5]:
crops = ['감자', '고구마', '고추(화건)', '깻잎', '녹두(국산)', '단감', '당근(무세척)', '대파', '들깨(국산)',
       '딸기', '무', '미나리', '방울토마토', '배(신고)', '배추', '백참깨(국산)', '복숭아(백도)',
       '사과(후지)', '생강(국산)', '수박', '시금치', '애호박', '양배추', '양파', '오이(가시)', '적상추',
       '참외', '콩(흰콩국산)', '토마토', '팥(국산)', '포도(캠벨)', '풋고추']
id2crop = {i: crop for i, crop in enumerate(crops)}

In [6]:
df['crop'] = df['crop_id'].apply(lambda x: id2crop[x])
df

,crop_id,연월,year,month,price,pred_price,crop
0,0,2001_1,2001,1,12191.0,12671.840043,감자
1,0,2001_2,2001,2,11558.0,14223.632715,감자
2,0,2001_3,2001,3,11885.0,14658.669814,감자
3,0,2001_4,2001,4,12133.0,16048.912606,감자
4,0,2001_5,2001,5,22625.0,16032.049901,감자
...,...,...,...,...,...,...,...
8059,31,2021_8,2021,8,NaN,51229.414671,풋고추
8060,31,2021_9,2021,9,NaN,70389.648956,풋고추
8061,31,2021_10,2021,10,NaN,40482.972822,풋고추
8062,31,2021_11,2021,11,NaN,51179.942012,풋고추


In [7]:
from collections import defaultdict
insurance = defaultdict(dict)
TARGET_YEAR = [2017, 2018, 2019, 2020, 2021]

for id_ in df.crop_id.unique():
    cur = df[df.crop_id == id_]
    cur_crop = id2crop[id_]
    
    # 연도별 가격 평균 집계
    cur_group = cur.groupby(['year']).agg({'price': 'mean', 'pred_price': 'mean'}).reset_index().set_index('year')

    # 5년 이동평균
    cur_group['MA_5_true'] = cur_group['price'].transform(lambda x: x.shift(1).rolling(5).mean())
    cur_group['MA_5_pred'] = cur_group['pred_price'].transform(lambda x: x.shift(1).rolling(5).mean())

    cur_group = cur_group.iloc[5:, :] # 5년의 이동평균을 구하면서 처음 5년의 데이터는 사용할 수 없음
    
    # 보험금지급여부
    cur_group['보험지급여부_true'] = np.where(cur_group['MA_5_true'] >= cur_group['price'], 1, 0)
    cur_group['보험지급여부_pred'] = np.where(cur_group['MA_5_pred'] >= cur_group['pred_price'], 1, 0)

    cur_group['빈도_mean_true'] = cur_group['보험지급여부_true'].expanding().mean()
    cur_group['빈도_mean_pred'] = cur_group['보험지급여부_pred'].expanding().mean()

    cur_group['심도_true'] = np.where(cur_group['보험지급여부_true'] == 1, cur_group['MA_5_true'] - cur_group['price'], 0)
    cur_group['심도_pred'] = np.where(cur_group['보험지급여부_pred'] == 1, cur_group['MA_5_pred'] - cur_group['pred_price'], 0)

    cur_group['심도_mean_true'] = cur_group['심도_true'].expanding().mean()
    cur_group['심도_mean_pred'] = cur_group['심도_pred'].expanding().mean()

    cur_group['보험료_true'] = cur_group['빈도_mean_true'] * cur_group['심도_mean_true']
    
    
    for target_year in TARGET_YEAR:
        premium = cur_group.at[target_year - 1, '보험료_true'] # 전년도까지의 보험료
        damage_true = cur_group.at[target_year, '심도_true'] # 해당연도의 손해액(true)
        damage_pred = cur_group.at[target_year, '심도_pred'] # 해당연도의 손해액(pred)
        insurance[target_year][cur_crop] = {
            '보험료': premium,
            '손해액_true': damage_true,
            '손해액_pred': damage_pred,
        }
cur_group

,price,pred_price,MA_5_true,MA_5_pred,보험지급여부_true,보험지급여부_pred,빈도_mean_true,빈도_mean_pred,심도_true,심도_pred,심도_mean_true,심도_mean_pred,보험료_true
year,,,,,,,,,,,,,
2006,44405.916667,47333.218362,33562.716667,37228.012818,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2007,39620.833333,41370.625689,36547.250000,38121.947328,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2008,36159.166667,36792.148213,37154.483333,38928.964752,1,1,0.333333,0.333333,995.316667,2136.816540,331.772222,712.272180,110.590741
2009,44918.250000,48072.634984,37727.383333,39493.227345,0,0,0.250000,0.250000,0.000000,0.000000,248.829167,534.204135,62.207292
2010,52317.500000,55482.091897,39857.516667,41950.591383,0,0,0.200000,0.200000,0.000000,0.000000,199.063333,427.363308,39.812667
2011,44199.666667,46609.233093,43484.333333,45810.143829,0,0,0.166667,0.166667,0.000000,0.000000,165.886111,356.136090,27.647685
2012,55643.000000,57284.365438,43443.083333,45665.346775,0,0,0.142857,0.142857,0.000000,0.000000,142.188095,305.259506,20.312585
2013,45447.416667,45935.415376,46647.516667,48848.094725,1,1,0.250000,0.250000,1200.100000,2912.679349,274.427083,631.186986,68.606771
2014,55208.000000,56615.184690,48505.166667,50676.748158,0,0,0.222222,0.222222,0.000000,0.000000,243.935185,561.055099,54.207819


In [8]:
insurance

defaultdict(dict,
            {2017: {'감자': {'보험료': 724.8044077134974,
               '손해액_true': 0.0,
               '손해액_pred': 0.0},
              '고구마': {'보험료': 335.70523415977874,
               '손해액_true': 0.0,
               '손해액_pred': 1744.4539640301446},
              '고추(화건)': {'보험료': 10205.769283746546,
               '손해액_true': 12607.199999999895,
               '손해액_pred': 107044.97053173633},
              '깻잎': {'보험료': 0.0,
               '손해액_true': 584.8166666666657,
               '손해액_pred': 478.5675968583164},
              '녹두(국산)': {'보험료': 10924.658402203851,
               '손해액_true': 0.0,
               '손해액_pred': 65298.705881969305},
              '단감': {'보험료': 692.9910468319563,
               '손해액_true': 0.0,
               '손해액_pred': 0.0},
              '당근(무세척)': {'보험료': 418.44504132231407,
               '손해액_true': 0.0,
               '손해액_pred': 0.0},
              '대파': {'보험료': 18.03099173553717,
               '손해액_true': 0.0,
               '손해액_p

In [9]:
with open('./results/crop_cluster.pickle', 'rb') as f:
    data = pickle.load(f)
data

{'감자': {'crop_class': '식량작물', 'price_class': 0},
 '고구마': {'crop_class': '식량작물', 'price_class': 4},
 '고추(화건)': {'crop_class': '채소', 'price_class': 3},
 '깻잎': {'crop_class': '채소', 'price_class': 2},
 '녹두(국산)': {'crop_class': '식량작물', 'price_class': 3},
 '단감': {'crop_class': '과일', 'price_class': 1},
 '당근(무세척)': {'crop_class': '채소', 'price_class': 0},
 '대파': {'crop_class': '채소', 'price_class': 4},
 '들깨(국산)': {'crop_class': '특용작물', 'price_class': 3},
 '딸기': {'crop_class': '채소', 'price_class': 2},
 '무': {'crop_class': '채소', 'price_class': 0},
 '미나리': {'crop_class': '채소', 'price_class': 0},
 '방울토마토': {'crop_class': '채소', 'price_class': 2},
 '배(신고)': {'crop_class': '과일', 'price_class': 4},
 '배추': {'crop_class': '채소', 'price_class': 0},
 '백참깨(국산)': {'crop_class': '특용작물', 'price_class': 4},
 '복숭아(백도)': {'crop_class': '과일', 'price_class': 1},
 '사과(후지)': {'crop_class': '과일', 'price_class': 4},
 '생강(국산)': {'crop_class': '채소', 'price_class': 4},
 '수박': {'crop_class': '채소', 'price_class': 2},
 '시금치': 

In [10]:
dfs = []

for year in TARGET_YEAR:
    statics = insurance[year]
    premium_total = defaultdict(int)
    damage_true_total = defaultdict(int)
    damage_pred_total = defaultdict(int)
    for crop in statics.keys():
        p_class = data[crop]['price_class']
        
        
        premium_total[p_class] += statics[crop]['보험료']
        damage_true_total[p_class] += statics[crop]['손해액_true']
        damage_pred_total[p_class] += statics[crop]['손해액_pred']
#     print(f'검정 연도: {year}')
#     print(f'cluster_id | 손해율(true) | 손해율(pred)')
#     for p_class in range(7):
#         print(f'{p_class} | {damage_true_total[p_class] / premium_total[p_class]:.4f} | {damage_pred_total[p_class] / premium_total[p_class]:.4f}')
    df = pd.DataFrame({
        '검정연도': [year] * 7,
        'cluster_id': range(7),
        '손해율(true)': [damage_true_total[i] / premium_total[i] for i in range(7)],
        '손해율(pred)': [damage_pred_total[i] / premium_total[i] for i in range(7)],
    })
    dfs.append(df)
    #print(f'\t손해율(pred): {damage_pred_total / premium_total}')
df_total = pd.concat(dfs, axis=0)
df_total

,검정연도,cluster_id,손해율(true),손해율(pred)
0,2017,0,0.298103,0.000000
1,2017,1,5.699854,9.440732
2,2017,2,2.310722,2.998792
3,2017,3,2.575596,7.993799
4,2017,4,3.719517,3.134554
5,2017,5,0.000000,0.000000
6,2017,6,4.413497,0.000000
0,2018,0,3.073802,1.660640
1,2018,1,0.524124,1.252409
2,2018,2,0.000000,0.000000


In [11]:
with open('./results/cluster_id.pkl', 'rb') as f:
    cluster_id = pickle.load(f)
cluster_id

defaultdict(list,
            {0: ['감자', '당근(무세척)', '무', '미나리', '배추', '양파'],
             4: ['고구마', '대파', '배(신고)', '백참깨(국산)', '사과(후지)', '생강(국산)', '팥(국산)'],
             3: ['고추(화건)', '녹두(국산)', '들깨(국산)', '콩(흰콩국산)'],
             2: ['깻잎', '딸기', '방울토마토', '수박', '애호박', '양배추', '오이(가시)', '토마토'],
             1: ['단감', '복숭아(백도)', '참외', '풋고추'],
             6: ['시금치', '적상추'],
             5: ['포도(캠벨)']})

In [12]:
df_total[df_total['cluster_id'] == 0]

,검정연도,cluster_id,손해율(true),손해율(pred)
0,2017,0,0.298103,0.000000
0,2018,0,3.073802,1.660640
0,2019,0,9.824884,9.133482
0,2020,0,2.540582,5.658598
0,2021,0,0.000000,8.578268


In [13]:
df_total[df_total['cluster_id'] == 1]

,검정연도,cluster_id,손해율(true),손해율(pred)
1,2017,1,5.699854,9.440732
1,2018,1,0.524124,1.252409
1,2019,1,1.729579,2.028327
1,2020,1,0.000000,0.000000
1,2021,1,0.000000,1.688885


In [14]:
df_total[df_total['cluster_id'] == 2]

,검정연도,cluster_id,손해율(true),손해율(pred)
2,2017,2,2.310722,2.998792
2,2018,2,0.000000,0.000000
2,2019,2,4.505322,5.217413
2,2020,2,0.301157,1.311041
2,2021,2,0.000000,3.583036


In [15]:
df_total[df_total['cluster_id'] == 3]

,검정연도,cluster_id,손해율(true),손해율(pred)
3,2017,3,2.575596,7.993799
3,2018,3,1.633395,4.692225
3,2019,3,0.000000,0.029593
3,2020,3,0.000000,0.000000
3,2021,3,0.000000,3.696351


In [16]:
df_total[df_total['cluster_id'] == 4]

,검정연도,cluster_id,손해율(true),손해율(pred)
4,2017,4,3.719517,3.134554
4,2018,4,1.763848,1.086086
4,2019,4,1.000356,0.747714
4,2020,4,0.000000,0.411568
4,2021,4,0.000000,0.231926


In [17]:
df_total[df_total['cluster_id'] == 5]

,검정연도,cluster_id,손해율(true),손해율(pred)
5,2017,5,0.000000,0.000000
5,2018,5,0.000000,0.000000
5,2019,5,0.275106,0.842832
5,2020,5,0.000000,0.000000
5,2021,5,0.000000,6.110964


In [18]:
df_total[df_total['cluster_id'] == 6]

,검정연도,cluster_id,손해율(true),손해율(pred)
6,2017,6,4.413497,0.000000
6,2018,6,0.000000,0.000000
6,2019,6,0.000000,16.860463
6,2020,6,5.985273,30.436355
6,2021,6,0.000000,28.923879


In [19]:
from collections import defaultdict

cases = defaultdict(dict)
years = [2017, 2018, 2019, 2020]

def pick_cluster(idx, items=[], n_pick=0):
    if n_pick == 3:
        cases[tuple(items)] = {}
        for year in years:
            df = df_total[df_total['cluster_id'].isin(items)]
            df = df[df.검정연도 == year]
            cases[tuple(items)][year] = {
                '손해율(true)_mean': df['손해율(true)'].mean(),
                '손해율(pred)_mean': df['손해율(pred)'].mean(),
                '손해율(true)_std': df['손해율(true)'].std(),
                '손해율(pred)_std': df['손해율(pred)'].std(),
            }
        return
    for i in range(idx + 1, 6 + 1):
        items.append(i)
        pick_cluster(i, items, n_pick + 1)
        items.pop()
        
pick_cluster(-1, items=[], n_pick=0)

In [20]:
for cluster in cases.keys():
    for year, data in cases[cluster].items():
        print(f'{cluster} | {year}')
        print(f"\t손해율(true)_mean: {data['손해율(true)_mean']:.4f}")
        print(f"\t손해율(true)_std : {data['손해율(true)_std']:.4f}")
        print(f"\t손해율(pred)_mean: {data['손해율(pred)_mean']:.4f}")
        print(f"\t손해율(pred)_std : {data['손해율(pred)_std']:.4f}")

(0, 1, 2) | 2017
	손해율(true)_mean: 2.7696
	손해율(true)_std : 2.7300
	손해율(pred)_mean: 4.1465
	손해율(pred)_std : 4.8239
(0, 1, 2) | 2018
	손해율(true)_mean: 1.1993
	손해율(true)_std : 1.6444
	손해율(pred)_mean: 0.9710
	손해율(pred)_std : 0.8653
(0, 1, 2) | 2019
	손해율(true)_mean: 5.3533
	손해율(true)_std : 4.1137
	손해율(pred)_mean: 5.4597
	손해율(pred)_std : 3.5588
(0, 1, 2) | 2020
	손해율(true)_mean: 0.9472
	손해율(true)_std : 1.3881
	손해율(pred)_mean: 2.3232
	손해율(pred)_std : 2.9620
(0, 1, 3) | 2017
	손해율(true)_mean: 2.8579
	손해율(true)_std : 2.7119
	손해율(pred)_mean: 5.8115
	손해율(pred)_std : 5.0846
(0, 1, 3) | 2018
	손해율(true)_mean: 1.7438
	손해율(true)_std : 1.2784
	손해율(pred)_mean: 2.5351
	손해율(pred)_std : 1.8793
(0, 1, 3) | 2019
	손해율(true)_mean: 3.8515
	손해율(true)_std : 5.2449
	손해율(pred)_mean: 3.7305
	손해율(pred)_std : 4.7847
(0, 1, 3) | 2020
	손해율(true)_mean: 0.8469
	손해율(true)_std : 1.4668
	손해율(pred)_mean: 1.8862
	손해율(pred)_std : 3.2670
(0, 1, 4) | 2017
	손해율(true)_mean: 3.2392
	손해율(true)_std : 2.7327
	손해율(pred)_mean: 4.1918
	손해율(pr

In [21]:
min_std_true, min_mean_true = {'cluster': -1, 'value': float('inf'), 'year': -1}, {'cluster': -1, 'value': float('inf'), 'year': -1}
min_std_pred, min_mean_pred = {'cluster': -1, 'value': float('inf'), 'year': -1}, {'cluster': -1, 'value': float('inf'), 'year': -1}

for cluster in cases.keys():
    for year, data in cases[cluster].items():
        if data['손해율(true)_mean'] < min_mean_true['value']:
            min_mean_true['value'] = data['손해율(true)_mean']
            min_mean_true['cluster'] = cluster
            min_mean_true['year'] = year
        
        if data['손해율(true)_std'] < min_std_true['value']:
            min_std_true['value'] = data['손해율(true)_std']
            min_std_true['cluster'] = cluster
            min_std_true['year'] = year
            
        if data['손해율(pred)_mean'] < min_mean_pred['value']:
            min_mean_pred['value'] = data['손해율(pred)_mean']
            min_mean_pred['cluster'] = cluster
            min_mean_pred['year'] = year
            
        if data['손해율(pred)_std'] < min_std_pred['value']:
            min_std_pred['value'] = data['손해율(pred)_std']
            min_std_pred['cluster'] = cluster
            min_std_pred['year'] = year

In [22]:
print(min_mean_true)
print(min_mean_pred)
print(min_std_true)
print(min_std_pred)

{'cluster': (1, 3, 4), 'value': 0.0, 'year': 2020}
{'cluster': (0, 5, 6), 'value': 0.0, 'year': 2017}
{'cluster': (1, 3, 4), 'value': 0.0, 'year': 2020}
{'cluster': (0, 5, 6), 'value': 0.0, 'year': 2017}


In [23]:
cases_v2 = defaultdict(dict)

def pick_cluster_v2(idx, items=[], n_pick=0):
    if n_pick == 3:
        df = df_total[df_total['cluster_id'].isin(items)]
        cases_v2[tuple(items)] = {
            '손해율(true)_mean': df['손해율(true)'].mean(),
            '손해율(pred)_mean': df['손해율(pred)'].mean(),
            '손해율(true)_std': df['손해율(true)'].std(),
            '손해율(pred)_std': df['손해율(pred)'].std(),
        }
        return
    for i in range(idx + 1, 6 + 1):
        items.append(i)
        pick_cluster_v2(i, items, n_pick + 1)
        items.pop()
        
pick_cluster_v2(-1, items=[], n_pick=0)

In [24]:
min_data = {
    '손해율(true)_mean': {'cluster': -1, 'value': float('inf')},
    '손해율(true)_std': {'cluster': -1, 'value': float('inf')},
    '손해율(pred)_mean': {'cluster': -1, 'value': float('inf')},
    '손해율(pred)_std': {'cluster': -1, 'value': float('inf')},
}

for cluster in cases_v2.keys():
    for k, v in cases_v2[cluster].items():
        if v < min_data[k]['value']:
            min_data[k]['value'] = v
            min_data[k]['cluster'] = cluster
            
min_data

{'손해율(true)_mean': {'cluster': (3, 4, 5), 'value': 0.7311878191591369},
 '손해율(true)_std': {'cluster': (3, 4, 5), 'value': 1.1742626964319842},
 '손해율(pred)_mean': {'cluster': (2, 4, 5), 'value': 1.7117283700971797},
 '손해율(pred)_std': {'cluster': (2, 4, 5), 'value': 2.011794305380933}}

In [25]:
for cluster in cases_v2.keys():
    print(cluster)
    for k, v in cases_v2[cluster].items():
        print(f'\t{k} {v:.4f}')

(0, 1, 2)
	손해율(true)_mean 2.0539
	손해율(pred)_mean 3.5034
	손해율(true)_std 2.8123
	손해율(pred)_std 3.3392
(0, 1, 3)
	손해율(true)_mean 1.8600
	손해율(pred)_mean 3.7236
	손해율(true)_std 2.7475
	손해율(pred)_std 3.5962
(0, 1, 4)
	손해율(true)_mean 2.0116
	손해율(pred)_mean 3.0035
	손해율(true)_std 2.7454
	손해율(pred)_std 3.4425
(0, 1, 5)
	손해율(true)_mean 1.5977
	손해율(pred)_mean 3.0930
	손해율(true)_std 2.8038
	손해율(pred)_std 3.6253
(0, 1, 6)
	손해율(true)_mean 2.2726
	손해율(pred)_mean 7.7108
	손해율(true)_std 3.0012
	손해율(pred)_std 10.1589
(0, 2, 3)
	손해율(true)_mean 1.8042
	손해율(pred)_mean 3.6369
	손해율(true)_std 2.6555
	손해율(pred)_std 3.2042
(0, 2, 4)
	손해율(true)_mean 1.9559
	손해율(pred)_mean 2.9169
	손해율(true)_std 2.6567
	손해율(pred)_std 3.0086
(0, 2, 5)
	손해율(true)_mean 1.5420
	손해율(pred)_mean 3.0063
	손해율(true)_std 2.7079
	손해율(pred)_std 3.2187
(0, 2, 6)
	손해율(true)_mean 2.2169
	손해율(pred)_mean 7.6241
	손해율(true)_std 2.9256
	손해율(pred)_std 10.0637
(0, 3, 4)
	손해율(true)_mean 1.7620
	손해율(pred)_mean 3.1370
	손해율(true)_std 2.5801
	손해율(pred)_std 3.333